# CycleGAN 莫奈作品
## 引入

In [ ]:
import os
import matplotlib.pyplot as plt

from models import CycleGAN
from utils import DataLoader

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

## 环境参数

In [ ]:
# run params
SECTION = 'paint'
RUN_ID = '0002'
DATA_NAME = 'monet2photo'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.makedirs(RUN_FOLDER)
    os.makedirs(os.path.join(RUN_FOLDER, 'viz'))
    os.makedirs(os.path.join(RUN_FOLDER, 'images'))
    os.makedirs(os.path.join(RUN_FOLDER, 'weights'))

mode = 'build'

## 加载数据

In [ ]:
IMAGE_SIZE = 256
data_loader = DataLoader(dataset_name=DATA_NAME, img_res=(IMAGE_SIZE, IMAGE_SIZE))

## 神经网络结构

In [ ]:
gan = CycleGAN(
    input_dim=(IMAGE_SIZE, IMAGE_SIZE, 3),
    learning_rate=0.0002,
    lambda_validation=1,
    lambda_reconstr=10,
    lambda_id=5,
    generator_type='resnet',
    gen_n_filters=32,
    disc_n_filters=64
)

if mode == 'build':
    gan.save(RUN_FOLDER)
else:
    gan.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))


In [ ]:
gan.g_BA.summary()

In [ ]:
gan.g_AB.summary()

In [ ]:
gan.d_A.summary()

In [ ]:
gan.d_B.summary()

## 训练CycleGAN

In [ ]:
BATCH_SIZE = 1
EPOCHS = 100
PRINT_EVERY_N_BATCHES = 300

TEST_A_FILE = '00490.jpg'
TEST_B_FILE = '2014-08-02 15_56_41.jpg'

gan.train(
    data_loader,
    run_folder=RUN_FOLDER,
    epochs=EPOCHS,
    test_A_file=TEST_A_FILE,
    test_B_file=TEST_B_FILE,
    batch_size=BATCH_SIZE,
    sample_interval=PRINT_EVERY_N_BATCHES
)

## 训练损失可视化

In [ ]:
fig = plt.figure(figsize=(20, 10))

plt.plot([x[1] for x in gan.g_losses], color='green',
         linewidth=0.1)  # DISCRIM LOSS
# plt.plot([x[2] for x in gan.g_losses], color='orange', linewidth=0.1)
plt.plot([x[3] for x in gan.g_losses],
         color='blue', linewidth=0.1)  # CYCLE LOSS
# plt.plot([x[4] for x in gan.g_losses], color='orange', linewidth=0.25)
plt.plot([x[5] for x in gan.g_losses], color='red', linewidth=0.25)  # ID LOSS
# plt.plot([x[6] for x in gan.g_losses], color='orange', linewidth=0.25)

plt.plot([x[0] for x in gan.g_losses], color='black', linewidth=0.25)

# plt.plot([x[0] for x in gan.d_losses], color='black', linewidth=0.25)

plt.xlabel('batch', fontsize=18)
plt.ylabel('loss', fontsize=16)

plt.ylim(0, 5)

plt.show()
